# Amazon SageMaker Model Bias Monitors: Fairness und Erklärbarkeit mit SageMaker Clarify

Amazon SageMaker Clarify hilft bei der Verbesserung Ihrer Machine Learning-Modelle, indem es potenzielle Tendenzen erkennt und dabei hilft, zu erklären, wie diese Modelle Vorhersagen treffen. Die von SageMaker Clarify bereitgestellten Funktionen für Fairness und Erklärbarkeit sind ein weiterer Schritt, der es AWS-Kunden erleichtert, vertrauenswürdige und verständliche Modelle für Machine Learning zu erstellen. Mit dem Produkt erhalten Sie Tools, die Sie bei folgenden Aufgaben unterstützen.

- Messung von Tendenzen, die in jeder Phase des ML-Lebenszyklus auftreten können (Datenerfassung, Modelltraining und -abstimmung sowie Überwachung von ML-Modellen, die für Inferenzen eingesetzt werden).
- Erstellung von Governance-Berichten zu Modellen für Risiko- und Compliance-Teams und externe Aufsichtsbehörden.
- Bereitstellung von Erklärungen zu Daten, Modellen und Überwachung, die zur Bewertung der Vorhersagen verwendet werden.

## Imports

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Konfiguration für diese Aufgabe

In [ ]:
# Ersetzen Sie unten <bucket_name> durch den Wert des modelArtifactBucket auf der linken Seite der Anweisungsseite
bucket = '<bucket_name>'

# Endpunktname der Produktion abrufen
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Modellname der Produktion abrufen
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### Dateien

Dateien lesen

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Aktivieren Sie Datenerfassung auf Ihrem Endpunkt

Um die Datenerfassung für Überwachungsaufträge zu aktivieren, geben Sie hier die neue Erfassungsoption namens DataCaptureConfig an. Sie ermöglicht die Erfassung der Nutzlast der Anfrage sowie der Antwort des Endpunkts. Die Erfassungskonfiguration wird auf alle Varianten angewendet. 

Dieser Prozess dauert in der Regel 10-15 Minuten.

**Hinweis:** Dieser Prozess kann über cloudformation automatisiert werden (Reviewer - bitte vorschlagen)

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Jetzt ist es Zeit, die neue Konfiguration anzuwenden und darauf zu warten, dass sie angewendet wird
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Endpunkt mit Beispiel-Traffic testen

In [ ]:
# Hier wählen wir eine zufällige Teilmenge von Daten aus den Testdatensätzen aus.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Aufrufen des Endpunkts mit Testdaten für einen Schnelltest
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    für Reihe in f:
        drucken(".", end="", flush=Wahr)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Erfasste Daten anzeigen

Listen Sie nun die Dateien mit den erfassten Daten auf, die in Amazon S3 gespeichert sind. Sie sollten sich darauf einstellen, verschiedene Dateien aus verschiedenen Zeiträumen zu sehen, die nach der Stunde des jeweiligen Aufrufs geordnet sind. Das Format des Amazon S3-Pfads lautet:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "Warten auf Laden der Datei..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        continue
print("Erfasste Dateien gefunden:")
print("\n ".join(capture_files))

Sehen Sie sich als nächstes den Inhalt einer einzelnen erfassten Datei an. Hier sollten Sie alle Daten sehen, die in einer spezifischen Datei in JSON-Zeilen-Format für Amazon Sagemaker erfasst wurden. Werfen Sie einen kurzen Blick auf die ersten Zeilen der erfassten Datei.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Zum Schluss wird der Inhalt einer einzelnen Zeile unten in einer formatierten JSON-Datei dargestellt, damit Sie einen besseren Überblick haben.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Amazon Sagemaker Clarify Setup

**Hinweis:** Die unten aufgeführten Beispielcodes stammen aus [Beispielcode](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb), bereitgestellt von Amazon Sagemaker. Manche Codes werden für die Voraussetzungen der Übung aktualisiert.

In [ ]:
# Sagemaker Clarifiy Prozessor mit folgendem Befehl ausführen
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Laden Sie die Test- und Übungsdatensätze in S3 hoch. Die _train_uri_ and _test_uri_ werden verwendet, um in den nächsten Schritten Daten direkt aus der S3-Schreibweise dataconfig und modelconfig zu ziehen.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### Tendenzen erkennen

Mit SageMaker Clarify können Sie mögliche Tendenzen vor und nach dem Training durch eine Vielzahl von Metriken erkennen.

### DataConfig und ModelConfig schreiben

Ein **DataConfig**-Objekt übermittelt einige grundlegende Informationen über Daten-I/O an SageMaker Clarify. Sie geben an, wo der Eingabedatensatz zu finden ist und wo die Ausgabe gespeichert wird sowie die Zielspalte (Label), die Namen der Überschriften und den Typ des Datensatzes.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Ein **ModelConfig**-Objekt übermittelt Informationen über Ihr trainiertes Modell. Um zusätzlichen Datenverkehr auf Ihren Produktionsmodellen zu vermeiden, erstellt und löscht SageMaker Clarify bei der Verarbeitung einen eigenen Endpunkt.

- **instance_type** and **instance_count** Geben Sie Ihren bevorzugten Instanztyp und die Instanzanzahl an, auf denen Ihr Modell während der Verarbeitung durch SageMaker Clarify ausgeführt werden soll. Der Test-Datensatz ist klein, so dass eine einzige Standardinstanz ausreicht, um dieses Beispiel durchzuführen. Wenn Sie einen großen, komplexen Datensatz haben, möchten Sie vielleicht einen besseren Instanztyp verwenden, um die Geschwindigkeit zu erhöhen, oder mehr Instanzen hinzufügen und so die Spark-Parallelisierung zu ermöglichen.

- **accept_type** legt das Nutzlastformat der Endpunktantwort fest und **content_type** legt das Nutzlastformat der Anfrage zum Endpunkt fest.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

Ein **ModelPredictedLabelConfig** liefert Informationen zum Format Ihrer Prognose. Das Random-Forest-Modell gibt Wahrscheinlichkeiten von Stichproben aus, so dass SageMaker Clarify den Endpunkt aufruft und dann probability_threshold verwendet, um die Wahrscheinlichkeit in binäre Labels für die Tendenzanalyse zu konvertieren. Eine Vorhersage, die über dem Schwellenwert liegt, wird als Labelwert 1 interpretiert, und eine Vorhersage, die darunter liegt oder gleich ist, als Labelwert 0 .

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### BiasConfig schreiben
SageMaker Clarify benötigt auch Informationen über die sensiblen Spalten (Facetten), die möglichen sensiblen Merkmale (facet_values_or_threshold) und die gewünschten Ergebnisse (label_values_or_threshold). SageMaker Clarify kann sowohl kategorische als auch kontinuierliche Daten für facet_values_or_threshold und label_values_or_threshold verarbeiten. In diesem Fall verwenden wir kategorische Daten.

Wir legen diese Informationen in der BiasConfig API fest. Hier lautet das positive Ergebnis Churn = 0, _Account Length_ ist eine sensible Kategorie und _Day Calls_ respondents sind die sensible Gruppe. group_name wird verwendet, um Unterkategorien für die Messung zu erstellen.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Tendenz vor dem Training**

Tendenzen können bereits in Ihren Daten enthalten sein bevor Sie ein Modelltraining durchführen. Die Überprüfung Ihrer Daten auf Tendenzen vor Beginn des Trainings kann dazu beitragen, Lücken in der Datenerfassung zu erkennen, Ihre Feature-Engineering-Abteilung zu informieren und zu verstehen, welche gesellschaftlichen Tendenzen die Daten widerspiegeln könnten.

Für die Berechnung von Tendenzmetriken vor dem Training ist kein trainiertes Modell erforderlich.

**Tendenz nach dem Training**

Für die Berechnung von Tendenzmetriken nach dem Training ist ein trainiertes Modell erforderlich.

Nicht beeinflusste Trainingsdaten (gemäß den Konzepten der Fairness, die mit Tendenzmetriken gemessen werden) können nach dem Training immer noch zu beeinflussten Modellprognosen führen. Ob dies der Fall ist, hängt von mehreren Faktoren ab, darunter die Wahl der Hyperparameter.

Sie können diese Optionen getrennt mit run_pre_training_bias() und run_post_training_bias() oder gleichzeitig mit run_bias() ausführen, wie unten dargestellt.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Tendenz-Bericht anzeigen**

In Studio können Sie die Ergebnisse unter der Registerkarte Experimente ansehen.

<img src="./recordings/bias_report.gif">

Jede Tendenzmetrik enthält ausführliche Erläuterungen mit Beispielen, die Sie erkunden können.

<img src="./recordings/bias_detail.gif">

Sie können die Ergebnisse auch in einer handlichen Tabelle zusammenfassen!

<img src="./recordings/bias_report_chart.gif">



Falls Sie noch kein Studio-Benutzer sind, können Sie den Tendenz-Bericht auch in pdf-, html- und ipynb-Format im folgenden S3-Bucket abrufen:

In [ ]:
bias_report_output_path

### Prognosen erläutern

Immer mehr geschäftliche Anforderungen und gesetzliche Vorschriften verlangen Erklärungen dafür, weshalb ein Modell die Entscheidung getroffen hat, die es getroffen hat. SageMaker Clarify verwendet SHAP, um den Beitrag, den jedes Eingabemerkmal zur endgültigen Entscheidung leistet, zu erläutern.

Der Kernel SHAP Algorithmus erfordert eine Basis (auch Background-Datensatz genannt). Der Basisdatensatz muss mit dem dataset_type von DataConfig übereinstimmen, und die Basisproben dürfen nur Funktion enthalten. Per Definition sollte die Basis entweder ein S3-URI zur Datei des Basisdatensatzes oder eine Liste von Proben an Ort und Stelle sein. In diesem Fall haben wir uns für Letzteres entschieden und die erste Probe des Testdatensatzes in die Liste aufgenommen.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Erklärbarkeitsbericht anzeigen
Wie beim Tendenz-Bericht können Sie auch den Erklärbarkeitsbericht in Studio unter der Registerkarte Experimente anzeigen


<img src="./recordings/explainability_detail.gif">

Die Registerkarte Modellerkenntnisse enthält direkte Links zu den Erkenntnissen über Berichte und Modelle.

Falls Sie noch kein Studio-Benutzer sind, können Sie diesen Bericht genau wie den Tendenz-Bericht auch im folgenden S3-Bucket abrufen.

In [ ]:
explainability_output_path

### Bereinigen

Vergessen Sie nicht, die Ressourcen, die wir für diese Demo eingerichtet und verwendet haben, am Ende zu bereinigen!

In [ ]:
session.delete_model(model_name)